# Run BEHAV3D feature extraction and T cell analysis for Imaris

This notebook runs the feature extraction and behavioral analysis pipeline of BEHAV3D

***This is specifically for processing with IMARIS***

To work correctly, it requires the following input:
- A metadata ***.csv*** containing information on every sample to be processed
- A ***.csv*** containing the position of the tracks
- Three feature ***.csv*** files containing information on dead dye intensity, organoid distance and tcell distance

In [1]:
from behav3d.preprocessing.csv.imaris_preprocessing import run_imaris_preprocessing
from behav3d.analysis.feature_extraction import (
    calculate_track_features,
    filter_tracks,
    summarize_track_features
)
from behav3d.analysis.tcell_analysis import run_tcell_analysis
from behav3d.analysis.backprojection import backproject_behav3d

import yaml
import pandas as pd

/Users/samdeblank/opt/anaconda3/envs/behav3d/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


<Figure size 100x100 with 0 Axes>

### Imaris preprocessing
Imaris outputs their calculated statistics in a certain .csv format that is not yet compatible with BEHAV3D processing. <br>
You therefore have to preprocess the resulting CSVs before applying BEHAV3D analysis.

Additionally, as Imaris uses surfaces that can not be extracted as voxel-based segments, the features "organoid contact", "t-cell contact" and "dead dye mean" can not be calculated outside of Imaris and thus must be calculated and extracted in Imaris

The following code preprocesses the Imaris segments based on their:
- centroid position (zyx)
- The distance of each T-cell to the nearest organoid 
- The distance of each T-cell to the nearest other T-cell
- The statistics of the dead dye mean, which is based on a dead dye channel

In [2]:
imaris_positions_csv = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/aim_dp1_exp13_img004_dietcart_n3_lg_car2_CD4_Position.csv"
imaris_dead_dye_mean_csv= "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/aim_dp1_exp13_img004_dietcart_n3_lg_car2_CD4_Intensity_Mean_Ch=5_Img=1.csv"
imaris_organoid_distance_csv = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/aim_dp1_exp13_img004_dietcart_n3_lg_car2_CD4_Shortest_Distance_to_Surfaces_Surfaces=Leukemia.csv"
imaris_tcell_distance_csv = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/aim_dp1_exp13_img004_dietcart_n3_lg_car2_CD4_Shortest_Distance_to_Surfaces_Surfaces=CD8.csv"

# output_path can be left empty to automatically name the output 
# and put it in the same folder as "imaris_positions_csv"
processed_imaris_output_path = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/df_imaris_preprocessed.csv"

run_imaris_preprocessing(
    df_positions_path=imaris_positions_csv,
    df_organoid_distances_path=imaris_organoid_distance_csv,
    df_tcell_distances_path=imaris_tcell_distance_csv,
    df_dead_dye_means_path=imaris_dead_dye_mean_csv,
    output_path=processed_imaris_output_path
)

### Set BEHAV3D parameters

***NOTE*** 

Dynamic Time Warping seems to work best if all tracks are of equal length. To get tracks to equal length, set both ***tcell_min_track_length*** and ***tcell_max_track_length*** to the same value

In [3]:
### Path for the output files generated by the pipeline
output_dir = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/BEHAV3D_run/behav3d_output"
### Path to the metadata csv that contains experiment specific information
metadata_csv_path = "/Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/BEHAV3D_run/metadata.csv"

#####################################################
################ Filter parameters ##################
#####################################################

### Maximum length of experiment (all timepoints this timepoint after will be removed)
######### Change this if your quality of imaging significantly reduces after a certain timepoint
tcell_exp_duration = 300
### Minimum track length to take into analysis. Filters out tracks shorter than this value
######### For example removes tracks for segmentation or tracking errors
tcell_min_track_length = 30
### Maximum track length to take into analysis. Cut longer tracks to the length defined below
######### Shortens tracks so that tracks of same length could be compared 
tcell_max_track_length = 100

#########################################################
### (Optional) parameters for behavioural clustering  ###
#########################################################

### Decrease for more resolution in the UMAP
umap_minimal_distance = 0.1
### Increase if you have a larger number of cells
umap_n_neighbors = 5
### Change to decrease/increase the number of clusters 
nr_of_clusters = 6

### The features to used in the Dynamic Time Warping to compare the tracks
dtw_features = [
    "z_mean_square_displacement", 
    "z_speed", 
    "z_mean_dead_dye", 
    "tcell_contact", 
    "organoid_contact"
    ]


### Read in the metadata file and display it
metadata = pd.read_csv(metadata_csv_path)
print("Metadata provided:")
metadata

output_dir: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/
metadata_csv_path: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/metadata.csv
imaris: True
ilastik_path: /Applications/ilastik-1.4.0b27-OSX.app/Contents/ilastik-release/run_ilastik.sh
ilastik_pixel_classifier_model: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_pix_clas_v5.ilp
ilastik_organoid_segmentation_model: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_obj_clas_v4.ilp
ilastik_organoid_postprocessing_model: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_faketrack_obj_split_v4.ilp
ilastik_tcell_segmentation_model: /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/ilastik_pipelines/AIM_MB2_obj_clas_TCELL.ilp
ilastik_tcell_postprocessing_model: /Us

,sample_name,organoid_line,tcell_line,exp_nr,well,dead_dye_channel,dead_dye_threshold,contact_threshold,pixel_distance_xy,pixel_distance_z,distance_unit,time_interval,time_unit,image_path,image_internal_path,tcell_track_csv
0,AIM_MB2_Exp58_Img003_donor899,10T,CART_WT1,1,well0,3,3000,3.6,3.54,1.2,µm,2,m,/Users/samdeblank/Documents/1.projects/BHVD_BE...,/image,/Users/samdeblank/Documents/1.projects/BHVD_BE...


### Check correct usage of metadata.csv

Checks the supplied metadata .csv for:
- Are all columns filled in?
- Do the paths supplied per row exist?

In [ ]:
assert not any(metadata.isna().any()), "Some column values have not been supplied. Make sure you correctly supply values for all columns in the metadata .csv"

for rowidx, sample_metadata in metadata.iterrows():
    sample_name = sample_metadata['sample_name']
    assert Path(sample_metadata["raw_image_path"]).exists(), f"The image_path supplied for Row {rowidx+1} '{sample_name}' does not exist"
    assert Path(sample_metadata["tcell_segments_path"]).exists(), f"The tcell_segments_path supplied for Row {rowidx+1} '{sample_name}' does not exist"
    assert Path(sample_metadata["organoid_segments_path"]).exists(), f"The organoid_segments_path supplied for Row {rowidx+1} '{sample_name}' does not exist"
    assert Path(sample_metadata["tcell_track_csv"]).exists(), f"The tcell_track_csv supplied for Row {rowidx+1} '{sample_name}' does not exist"
print("Metadata file is OK!")

---

# Calculate the track features, filter tracks and summarize track features

---

**calculate_track_features** <br>
Calculates movement, contact and intensity features for each timepoint in all tracks per experiment

**filter_tracks** <br>
Filter out tracks based on:
- Maximum experiment length (tcell_exp_duration)    
- Minimum track length (tcell_min_track_length)
- Tracks starting at timepoint 1 with a dead dye mean over the dead_dye_threshold (dead_dye_threshold)

Additonally, all tracks are cut down to:
- Maximum track length (tcell_max_track_length)

**summarize_track_features** <br>
Summarizes the features into one value for each TrackID per experiment

--------------------


## T cells

**Calculation of track features**

*Expected duration: ~30 minutes*

In [4]:
df_tracks = calculate_track_features(
    metadata=metadata, 
    output_dir=output_dir, 
    cell_type="tcells",
    imaris=True
    )

--------------- Processing tcells: AIM_MB2_Exp58_Img003_donor899 ---------------
###### Running track feature calculation
Performing feature calculation from Imaris processing..
- Loading in tracks csv...
- Calculating contact with organoids and other T cells... (From Imaris)
- Interpolating missing timepoints based on time interval
- Calculating cell death based on defined dead_dye_threshold 3000
- Converting distance and time unit to default µm and hours...
- Calculating movement features...
- Perform z-normalization on selected feature columns
- Writing output to /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/AIM_MB2_Exp58_Img003_donor899_tcells_track_features.csv
### DONE - elapsed time: 0:00:05

--------------- Filtering tracks ---------------
- Writing filtered tracks to /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/BEHAV3D_combined_track_features_filtered.csv


**Filtering tracks and summarizing features**

*Expected duration: <1 minute*

In [ ]:
df_tracks=pd.read_csv(Path(output_dir, "analysis/tcells/track_features/BEHAV3D_combined_track_features.csv"))
df_tracks_filt = filter_tracks(
    df_tracks,
    metadata=metadata, 
    output_dir=output_dir, 
    tcell_exp_duration=tcell_exp_duration,
    tcell_min_track_length=tcell_min_track_length,
    tcell_max_track_length=tcell_max_track_length,
    cell_type="tcells"
    )
df_tracks_summ = summarize_track_features(
    df_tracks_filt, 
    output_dir=output_dir,
    cell_type="tcells")

## Organoids
*IN DEVELOPMENT*

In [5]:
df_tracks_clustered = run_tcell_analysis(config)

Set 'tcell_min_track_length' and 'tcell_max_track_length' to the same value to create equal tracks
- Calculating the dynamic time warping distance matrix
- Fitting the dynamic time warping to a UMAP


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


- Performing clustering on the UMAP data
- Writing clustered tracks to /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/BEHAV3D_UMAP_clusters.csv
- Producing clustered UMAP plots with displayed Track features
- Producing heatmaps with summarized cluster features
- Producing percentage plots of each cluster per combination of T-cell and organoid line
- Writing summarized tracks to /Users/samdeblank/Documents/1.projects/BHVD_BEHAV3D/BEHAV3D-ilastik/test/imaris_run/imaris_preprocess_test/BEHAV3D_UMAP_cluster_percentages.csv


In [ ]:
# df_tracks = calculate_track_features(
#     config, metadata, cell_type="organoids", imaris=True)

## T cell behavioral analysis

***Dynamic Time Warping*** <br>

Performs Dynamic Time Warping on the various tracks using various features to calculate a distance matrix between all tracks

Features used for dynamic time warping by default:
- z_mean_square_displacement
- z_speed
- z_mean_dead_dye
- tcell_contact
- organoid_contact

***UMAP fitting and clustering*** <br>

This distance matrix is fitted into a UMAP, and K-means clustering is used to cluster the tracks into clusters

Parameters for UMAP and clustering can be adjusted in the config file:
- umap_minimal_distance
- umap_n_neighbors
- nr_of_clusters

***Cluster features display*** <br>

The results of the clustering are displayed in multiple ways:
- The UMAP with clusters and features backprojected onto this UMAP (BEHAV3D_UMAP_clusters.csv)
- A heatmap containing the mean feature values for all tracks per cluster to check general behavior patterns (BEHAV3D_UMAP_cluster_feature_heatmap.pdf)
- A barplot containing the percentages of cells belonging to each cluster per Tcell type (rows) and organoid lines (columns)


In [ ]:
df_tracks_clustered = run_tcell_analysis(
    output_dir=output_dir,
    umap_minimal_distance=umap_minimal_distance,
    umap_n_neighbors=umap_n_neighbors,
    nr_of_clusters=nr_of_clusters,
    dtw_features= dtw_features
    )

---

# Backproject results

---

Visualize the cluster IDs and other features over the segments
- Optionally save it as a compressed .h5

If already saved as backprojection, it will not redo the backprojection but load in existing backprojection .h5

 <br>

**First time making a backprojection of all features**

*Duration: ~60 minutes*

**Loading in existing backprojection .h5**

*Duration: ~10 minutes*


In [ ]:
print("Samples in metadata .csv:")
for sample_name in metadata["sample_name"]:
    print(sample_name)

In [ ]:
sample_to_backproject="AIM_MB2_Exp58_Img003_donor899"

backproject_behav3d(
    sample_name=sample_to_backproject,
    metadata=metadata,
    output_dir=output_dir,
    save=True
)